# Day 19: Not Enough Minerals

A classic resource and machine building strategy game.

First up, let's have a quick check of the size of the problem:

24 Minutes, with a minute-resolution simulation.
Puzzle input contains 30 blueprints.

The blueprints vary the quantity of the source materials required, but they don't change the type of resource.

## Approach

A first glance, this looks similar to the valves puzzle from a few days ago. I solved that using a brute-force technique, checking every single possible option (and discarding some obvious losers along the way to improve performance).
But there are so many more possibilities here, that taking that approach seems wrong. And also a bit repetative... I want to try something different.

So... how about having a strategy engine that works backwards from the goal?

Generic recipes:
a Ore              -> ore bot   -> Ore
b Ore              -> clay bot  -> Clay
c Ore + d Clay     -> obs bot   -> Obsidian
e Ore + f Obsidian -> geode bot -> Geode

Re-arrange to work backwards: (and ignoring time)
1 Geode = e Ore + f Obsidian
        = e Ore + f (c Ore + d Clay)
        = e Ore + fc Ore + fdb Ore 
        = ( e + fc + fdb ) Ore

Start with 1 ore collecting Robot.
Ore is the primary contraint... but we don't want an ore stock-pile at the end.


Other approaches to consider... could this be turned into a single function that can be optimised? 
What would the inputs be? The choice how many robots of each type to build in each minute.
But it's integer choices... so we're into integer programming - maybe try using: https://www.cvxpy.org/examples/applications/OOCO.html

But perhaps I should try a slightly more straightforward way first?

In [1]:
testData ="""Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

import re

def process(input:str):
    pattern = '(\d+)'
    for l in input.splitlines():
        match = re.findall(pattern,l)
        blueprint = [int(i) for i in match]
        print(blueprint)

#test
process(testData)

[1, 4, 2, 3, 14, 2, 7]
[2, 2, 3, 3, 8, 3, 12]


In [12]:
ID = 'id'
ORE = 'ore'
CLAY = 'clay'
OBSIDIAN = 'obsidian'
GEODE = 'geoode'
RUNTIME = 24

class Puzzle:
    def blueprintsFrom(self, input):
        pattern = '(\d+)'
        blueprints = []
        for l in input.splitlines():
            match = re.findall(pattern,l)
            intarr = [int(i) for i in match]
            blueprint = {}
            blueprint[ID] = intarr[0]
            blueprint['a'] = intarr[1]
            blueprint['b'] = intarr[2]
            blueprint['c'] = intarr[3]
            blueprint['d'] = intarr[4]
            blueprint['e'] = intarr[5]
            blueprint['f'] = intarr[6]
            blueprints.append(blueprint)
        return blueprints
    
    def __init__(self, input:str):
        self.blueprints = self.blueprintsFrom(input)


class Game:
    def __init__(self, blueprint):
        self.time = 0
        self.materials= {ORE:0, CLAY:0, OBSIDIAN:0, GEODE:0}
        self.bots= {ORE:1, CLAY:0, OBSIDIAN:0, GEODE:0}
        self.blueprint = blueprint

    def playGame(self)->int:
        #play the game for a single blueprint, return the quality score
        print(self.blueprint)
        while self.time < RUNTIME:
            self.tick()

        return self.materials[OBSIDIAN] * self.blueprint[ID]

    def tick(self):
        #progres the game 1 minute
        self.time += 1
        startMaterials = self.materials.copy()
        startBots = self.bots.copy()
        self.buildBots(startMaterials)
        self.collectMaterials(startBots)
        print('T=' + str(self.time)+ ' Bots: ' + str(self.bots) + ' Materials: ' + str(self.materials))

    def buildBots(self, startMaterials):
        #this is where the strategy happens
        #there are four type of bots that could be built... need to work out which makes most sense.
        #NOTE: Only one robot can be build - this makes choices slightly easier!
        remainingTime = RUNTIME - self.time - 1 #minus 1 as bot doens't build anything this minute
        botAvailable = True
        
        #can we calculate an effective value of each material? i.e. how useful to have more of it?
        #should be the value relative to ORE: e.g. Supply over demand
        value = {}
        value[OBSIDIAN] = (self.blueprint['f'] - startMaterials[OBSIDIAN]) / (self.blueprint['e'] + self.blueprint['c'] + self.blueprint['b'] + self.blueprint['a'] - startMaterials[ORE])
        value[CLAY] = (self.blueprint['d'] - startMaterials[CLAY]) / (self.blueprint['e'] + self.blueprint['c'] + self.blueprint['b'] + self.blueprint['a'] - startMaterials[ORE])

        futureBotValue = {}
        futureBotValue[ORE] = (remainingTime * startMaterials[ORE]) / (self.blueprint['e'] + self.blueprint['c'] + self.blueprint['b'] + self.blueprint['a'] - startMaterials[ORE])
        futureBotValue[OBSIDIAN] = remainingTime * value[OBSIDIAN]
        futureBotValue[CLAY] = remainingTime * value[CLAY]

        print('FUTURE BOT VALUE: '+ str(futureBotValue))

        #the it gets trickier. It might not always make sense to make a obs bot.. e.g. if ore would be better used later.
        #so we need a way of projecting expected materials.
        #each bot effectively has a estimated future utility... how many choices do we actually have.

        #first part of the strategy is easy... if I can build a geode bot, then I should..
        if startMaterials[ORE] >= self.blueprint['e'] and startMaterials[OBSIDIAN] >= self.blueprint['f'] and botAvailable:
            #build a bot
            self.bots[GEODE] += 1
            botAvailable = False
            self.materials[ORE] -= self.blueprint['e']
            self.materials[OBSIDIAN] -= self.blueprint['f']
        

        if startMaterials[ORE] >= self.blueprint['c'] and startMaterials[CLAY] >= self.blueprint['d'] and botAvailable and futureBotValue[OBSIDIAN] > 1 and futureBotValue[OBSIDIAN] > futureBotValue[CLAY] and futureBotValue[OBSIDIAN] > futureBotValue[ORE]:
            #we could build a obsidian bot, but should we?
            #Yes, if the future projected utility if higher than the value of the materials
            self.bots[OBSIDIAN] += 1
            botAvailable = False
            self.materials[ORE] -= self.blueprint['c']
            self.materials[CLAY] -= self.blueprint['d']
        
        if startMaterials[ORE] >= self.blueprint['b'] and botAvailable and futureBotValue[CLAY] > 1 and futureBotValue[CLAY] > futureBotValue[ORE]: 
            self.bots[CLAY] += 1
            botAvailable = False
            self.materials[ORE] -= self.blueprint['b']
        
        if startMaterials[ORE] >= self.blueprint['a']  and botAvailable and futureBotValue[ORE] > 1:
            self.bots[ORE] += 1
            botAvailable = False
            self.materials[ORE] -= self.blueprint['a']


    
    def collectMaterials(self, startBots):
        for k,v in startBots.items():
            self.materials[k] += v


     


#tests
tp = Puzzle(testData)
#print(tp.blueprints)
tg = Game(tp.blueprints[0])
print(tg.playGame())



{'id': 1, 'a': 4, 'b': 2, 'c': 3, 'd': 14, 'e': 2, 'f': 7}
FUTURE BOT VALUE: {'ore': 0.0, 'obsidian': 14.0, 'clay': 28.0}
T=1 Bots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0}
FUTURE BOT VALUE: {'ore': 2.1, 'obsidian': 14.7, 'clay': 29.4}
T=2 Bots: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 2, 'clay': 0, 'obsidian': 0, 'geoode': 0}
FUTURE BOT VALUE: {'ore': 4.444444444444445, 'obsidian': 15.555555555555555, 'clay': 31.11111111111111}
T=3 Bots: {'ore': 1, 'clay': 1, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 1, 'clay': 0, 'obsidian': 0, 'geoode': 0}
FUTURE BOT VALUE: {'ore': 1.9, 'obsidian': 13.299999999999999, 'clay': 26.599999999999998}
T=4 Bots: {'ore': 1, 'clay': 1, 'obsidian': 0, 'geoode': 0} Materials: {'ore': 2, 'clay': 1, 'obsidian': 0, 'geoode': 0}
FUTURE BOT VALUE: {'ore': 4.0, 'obsidian': 14.0, 'clay': 26.0}
T=5 Bots: {'ore': 1, 'clay': 2, 'obsidian': 0, 'geoode': 0} Materials: {'